In [1]:
%matplotlib inline

import numpy as np 
import pandas as pd 
pd.set_option('display.max_rows',5)
import matplotlib.pyplot as plt
from matplotlib import pyplot

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

from xgboost import XGBClassifier
from xgboost import plot_importance

In [2]:
#Load the dataframes
df_seeds = pd.read_csv('NCAATourneySeeds.csv')
df_matches_NCAA = pd.read_csv('NCAATourneyCompactResults.csv')
df_matches_reg = pd.read_csv('RegularSeasonCompactResults.csv')

#We can choose to concatinate the two dataframes or only use one
#df_matches = pd.concat([df_matches_NCAA, df_matches_reg])
df_matches = df_matches_NCAA

In [3]:
#Display top 5 rows in the dataframe

print(df_matches_NCAA.shape)
print(df_matches_reg.shape)

print(df_seeds.head())
df_matches_NCAA.head()

(2117, 8)
(150684, 8)
   Season Seed  TeamID
0    1985  W01    1207
1    1985  W02    1210
2    1985  W03    1228
3    1985  W04    1260
4    1985  W05    1374


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [4]:
#Extract the numeric part of the seed (drop the regions at the beginning and the (a,b) at the end).
#Then convert the numeric string to integer
df_seeds['Seed'] = df_seeds['Seed'].str.extract('(\d+)').astype(int)
cols = ['Season', 'TeamID', 'Seed']
df_seeds = df_seeds[cols]
df_seeds.head()

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


,Season,TeamID,Seed
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [5]:
#Drop some of the columns to make the dataframe simpler
#DayNum is the day in which the match was played
#NumOT: the number of overtime periods in the game.
#WLoc is  this identifies the "location" of the winning team. 
# ---If the winning team was the home team, this value will be "H".
# ---If the winning team was the visiting team, this value will be "A".
# ---If it was played on a neutral court, then this value will be "N".

#axis = 0 drops a row. axis = 1 drops from the columns
#inplace = True does the operation inplace, so you dont need to write df = df.drop....
df_matches.drop(labels=['DayNum', 'WLoc', 'NumOT'], inplace = True, axis=1)
print(df_matches.shape)

# Drop the matches before some year if you want, or choose 1984 to take all data
df_matches = df_matches[df_matches.Season > 1984]
print(df_matches.shape)

#Swap the second and third columns to have teams and scores next to each other for better visualisation
df_matches = df_matches[["Season","WTeamID","LTeamID", "WScore", "LScore"]]
df_matches

(2117, 5)
(2117, 5)


,Season,WTeamID,LTeamID,WScore,LScore
0,1985,1116,1234,63,54
1,1985,1120,1345,59,58
...,...,...,...,...,...
2115,2017,1314,1332,77,76
2116,2017,1314,1211,71,65


In [6]:
#Gather the information from a team in each season.
#E.g in 2011, how many games Team 1117 won, how many lost, how many scored in the won games,
# Total of all scores in that season etc.

df_WTeams = pd.DataFrame({'GamesWon' : df_matches.groupby(["Season", "WTeamID"]).size(),
                       'WTotalScore' : df_matches.groupby(["Season", "WTeamID"])["WScore"].sum(),
                      'WScoreAvg' : df_matches.groupby(["Season", "WTeamID"])["WScore"].mean()}).reset_index()
df_WTeams = df_WTeams.rename(columns={'WTeamID':'TeamID'})

df_LTeams = pd.DataFrame({'GamesLost' : df_matches.groupby(["Season", "LTeamID"]).size(),
                       'LTotalScore' : df_matches.groupby(["Season", "LTeamID"])["LScore"].sum(),
                      'LScoreAvg' : df_matches.groupby(["Season", "LTeamID"])["LScore"].mean()}).reset_index()
df_LTeams = df_LTeams.rename(columns={'LTeamID':'TeamID'})

df_Teams = pd.merge(left=df_WTeams, right=df_LTeams, how='outer', on=['Season', 'TeamID'])
df_Teams = df_Teams.fillna(0)

df_Teams["TotalScore"] = df_Teams.WTotalScore + df_Teams.LTotalScore
df_Teams["AverageScore"] = (df_Teams.WTotalScore + df_Teams.LTotalScore)/(df_Teams.GamesWon+df_Teams.GamesLost)

df_Teams = df_Teams.round().astype('int')
df_Teams = pd.merge(left=df_seeds, right=df_Teams, how='right', on=['Season', 'TeamID'])
df_Teams = df_Teams.sort_values(by=['Season', 'Seed'])
df_Teams.head()

,Season,TeamID,Seed,GamesWon,WScoreAvg,WTotalScore,GamesLost,LScoreAvg,LTotalScore,TotalScore,AverageScore
0,1985,1207,1,5,67,333,1,64,64,397,66
16,1985,1385,1,4,76,306,1,59,59,365,73
32,1985,1328,1,3,86,257,1,61,61,318,80
48,1985,1276,1,1,59,59,1,55,55,114,57
1,1985,1210,2,3,65,196,1,54,54,250,62


In [7]:
# We are going to merge the matches and seeds databases.
# The seed and score difference in each match is also calculated
# First, in the seeds df, change TeamID to WTeamID and LTeamID to match with the df_matches df
# Then merge the winning teams seeds
# Then merge the losing teams seeds
# Calculate the seed and score difference in each match

df_winseeds = df_Teams.rename(columns={'TeamID':'WTeamID', 'Seed':'WSeed', 'GamesWon':'WGamesWon',
                                      'WScoreAvg':'WWScoreAvg', 'WTotalScore':'WWTotalScore',
                                     'GamesLost':'WGamesLost','LScoreAvg':'WLScoreAvg',
                                       'LTotalScore':'WLTotalScore',
                                     'AverageScore':'WAverageScore', 'TotalScore':'WTotalScore'})

df_lossseeds = df_Teams.rename(columns={'TeamID':'LTeamID', 'Seed':'LSeed', 'GamesWon':'LGamesWon',
                                      'WScoreAvg':'LWScoreAvg', 'WTotalScore':'LWTotalScore',
                                     'GamesLost':'LGamesLost','LScoreAvg':'LLScoreAvg',
                                       'LTotalScore':'LLTotalScore',
                                     'AverageScore':'LAverageScore', 'TotalScore':'LTotalScore'})

df_dummy = pd.merge(left=df_matches, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df = pd.merge(left=df_dummy, right=df_lossseeds, how='left', on=['Season', 'LTeamID'])
df['SeedDiff'] = df.WSeed - df.LSeed
df['GamesDiff'] = df.WGamesWon - df.LGamesWon
df['GamesDiff'] = df.WGamesWon - df.LGamesWon
df['ScoreDiff'] = df.WTotalScore - df.LTotalScore
df['AvgScoreDiff'] = df.WAverageScore - df.LAverageScore
df.head()

,Season,WTeamID,LTeamID,WScore,LScore,WSeed,WGamesWon,WWScoreAvg,WWTotalScore,WGamesLost,...,LWTotalScore,LGamesLost,LLScoreAvg,LLTotalScore,LTotalScore,LAverageScore,SeedDiff,GamesDiff,ScoreDiff,AvgScoreDiff
0,1985,1116,1234,63,54,9,1,63,63,1,...,0,1,54,54,54,54,1,1,74,10
1,1985,1120,1345,59,58,11,2,62,125,1,...,0,1,58,58,58,58,5,2,123,2
2,1985,1207,1250,68,43,1,5,67,333,1,...,0,1,43,43,43,43,-15,5,354,23
3,1985,1229,1425,58,55,9,1,58,58,1,...,0,1,55,55,55,55,1,1,72,9
4,1985,1242,1325,49,38,3,1,49,49,1,...,0,1,38,38,38,38,-11,1,75,18


In [9]:
df_TeamsA = pd.DataFrame({ 'TeamID' : df_Teams.TeamID.unique(),
                        'NQualified' : df_Teams.groupby(["TeamID"])["Season"].size(),
                       'Champ' : df_Teams[df_Teams.GamesLost ==0].groupby(["TeamID"])["Season"].size(),
                      'AvgWon' : df_Teams.groupby(["TeamID"])["GamesWon"].mean(),
                        'AvgSeed' : df_Teams.groupby(["TeamID"])["Seed"].mean()  
                         })
df_TeamsA = df_TeamsA.fillna(0)
df_TeamsA = df_TeamsA.sort_values(by='NQualified', ascending=False)
df_TeamsA.head()

,AvgSeed,AvgWon,Champ,NQualified,TeamID
TeamID,,,,,
1181,2.218750,2.843750,5.0,32,1299
1242,2.500000,2.500000,2.0,32,1371
1112,4.451613,1.741935,1.0,31,1301
1314,2.800000,2.933333,4.0,30,1418
1246,3.571429,2.785714,3.0,28,1258


In [10]:
df_winseeds = df_TeamsA.rename(columns={'TeamID':'WTeamID', 'AvgWon':'WAvgWon', 'Champ':'WChamp',
                                       'NQualified':'WNQualified', 'AvgSeed': 'WAvgSeed'})

df_lossseeds = df_TeamsA.rename(columns={'TeamID':'LTeamID', 'AvgWon':'LAvgWon', 'Champ':'LChamp',
                                       'NQualified':'LNQualified', 'AvgSeed': 'LAvgSeed'})

df_dummy = pd.merge(left=df_matches, right=df_winseeds, how='left', on='WTeamID')
df_dummy2 = pd.merge(left=df_dummy, right=df_lossseeds, how='left', on='LTeamID')


df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'Seed':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'Seed':'LSeed'})
df_dummy3 = pd.merge(left=df_dummy2, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df = pd.merge(left=df_dummy3, right=df_lossseeds, how='left', on=['Season', 'LTeamID'])

df['SeedDiff'] = df.WSeed - df.LSeed
df['AvgSeedDiff'] = df.WAvgSeed - df.LAvgSeed
df['ChampDiff'] = df.WChamp - df.LChamp
df['NQualifiedDiff'] = df.WNQualified - df.LNQualified
df['AvgWonDiff'] = df.WAvgWon - df.LAvgWon
df.head()

,Season,WTeamID,LTeamID,WScore,LScore,WAvgSeed,WAvgWon,WChamp,WNQualified,LAvgSeed,LAvgWon,LChamp,LNQualified,WSeed,LSeed,SeedDiff,AvgSeedDiff,ChampDiff,NQualifiedDiff,AvgWonDiff
0,1985,1116,1234,63,54,16.0,0.00,0.0,1,9.222222,0.388889,0.0,18,9,8,1,6.777778,0.0,-17,-0.388889
1,1985,1120,1345,59,58,11.5,0.25,0.0,4,15.000000,0.000000,0.0,5,11,6,5,-3.500000,0.0,-1,0.250000
2,1985,1207,1250,68,43,12.0,0.00,0.0,2,5.000000,0.000000,0.0,1,1,16,-15,7.000000,0.0,1,0.000000
3,1985,1229,1425,58,55,13.0,0.00,0.0,1,12.000000,0.500000,0.0,2,9,8,1,1.000000,0.0,-1,-0.500000
4,1985,1242,1325,49,38,14.2,0.40,0.0,5,9.300000,0.700000,0.0,10,3,14,-11,4.900000,0.0,-5,-0.300000


In [11]:
# Assign a win (Result = 1) to the seed and score differences
# Assign a lose (Result = 0) to the negative of the seed and score difference
# This gives us a relationship of how seed and score difference can be classified
# as win or lose. We expect higher seed difference to be more likely to win.


df_wins = df[['SeedDiff','AvgSeedDiff','ChampDiff', 'NQualifiedDiff', 'AvgWonDiff']]
df_wins['Result'] = 1

df_losses = -df[['SeedDiff','AvgSeedDiff','ChampDiff', 'NQualifiedDiff', 'AvgWonDiff']]
df_losses['Result'] = 0

data = pd.concat((df_wins, df_losses))
data.head()

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SeedDiff,AvgSeedDiff,ChampDiff,NQualifiedDiff,AvgWonDiff,Result
0,1.0,6.777778,0.0,-17.0,-0.388889,1
1,5.0,-3.500000,0.0,-1.0,0.250000,1
2,-15.0,7.000000,0.0,1.0,0.000000,1
3,1.0,1.000000,0.0,-1.0,-0.500000,1
4,-11.0,4.900000,0.0,-5.0,-0.300000,1


In [12]:
# The features of the model. 'Result' is the output so should subtract it
# Let's forget about ScoreDiff for now and only learn on SeedDiff

features = list(set(data.columns) - {'Result', 'ChampDiff', 'NQualifiedDiff', 'AvgSeedDiff', 'AvgWonDiff'})

X_train = data[features]
y_train = data.Result
#validation_data = data.iloc[1949:2117,:]
#X_val = validation_data[features]
features

['SeedDiff']

In [13]:
test_fold1 = np.pad([-1 for i in range(1849)], (0,2117-1849), 'constant', constant_values=(0))
test_fold = np.concatenate((test_fold1, test_fold1))
ps = PredefinedSplit(test_fold)
test_fold.shape

(4234,)

In [14]:
# Choose a model
# Choose a range for your parameter that want to GridSearch
# Train the model by fitting X and y
#Print the best parameter

logreg = LogisticRegression(penalty = 'l2', solver = 'saga')
params = {'C': np.linspace(start=0.001, stop=0.1, num=100)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True, cv=ps)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

Best log_loss: -0.5708, with best C: 0.001


In [15]:
#Now we neeed to use our trained model to predict the sample submission data 
# The format of Sample submission is SSSS_XXXX_YYYY
# S is season ID, X and Y are the two team ID's

df_sample = pd.read_csv('SampleSubmissionStage1.csv')
n_games = len(df_sample)
print(n_games)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

9112


In [16]:
df_sample2 = df_sample.copy()
df_sample2[['Season','Team1', 'Team2']] = df_sample['ID'].str.split('_',expand=True)
df_sample2 = df_sample2.drop(columns=['ID'], axis=1)
df_sample2.head()

,Pred,Season,Team1,Team2
0,0.5,2014,1107,1110
1,0.5,2014,1107,1112
2,0.5,2014,1107,1113
3,0.5,2014,1107,1124
4,0.5,2014,1107,1140


In [17]:
df_winseeds = df_TeamsA.rename(columns={'TeamID':'Team1', 'AvgWon':'AvgWon1', 'Champ':'Champ1',
                                       'NQualified':'NQualified1', 'AvgSeed': 'AvgSeed1'})
df_lossseeds = df_TeamsA.rename(columns={'TeamID':'Team2', 'AvgWon':'AvgWon2', 'Champ':'Champ2',
                                       'NQualified':'NQualified2', 'AvgSeed': 'AvgSeed2'})

df_sample2['Team1'] = df_sample2['Team1'].astype(int)
df_sample2['Team2'] = df_sample2['Team2'].astype(int)
df_sample2['Season'] = df_sample2['Season'].astype(int)

df_dummy = pd.merge(left=df_sample2, right=df_winseeds, how='left', on=['Team1'])
df_dummy2 = pd.merge(left=df_dummy, right=df_lossseeds, how='left', on='Team2')


df_winseeds = df_seeds.rename(columns={'TeamID':'Team1', 'Seed':'Seed1'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'Team2', 'Seed':'Seed2'})
df_dummy3 = pd.merge(left=df_dummy2, right=df_winseeds, how='left', on=['Season', 'Team1'])
df_test = pd.merge(left=df_dummy3, right=df_lossseeds, how='left', on=['Season', 'Team2'])

df_test['SeedDiff'] = df_test.Seed1 - df_test.Seed2
df_test['AvgSeedDiff'] = df_test.AvgSeed1 - df_test.AvgSeed2
df_test['ChampDiff'] = df_test.Champ1 - df_test.Champ2
df_test['NQualifiedDiff'] = df_test.NQualified1 - df_test.NQualified2
df_test['AvgWonDiff'] = df_test.AvgWon1 - df_test.AvgWon2
df_test.head()

,Pred,Season,Team1,Team2,AvgSeed1,AvgWon1,Champ1,NQualified1,AvgSeed2,AvgWon2,Champ2,NQualified2,Seed1,Seed2,SeedDiff,AvgSeedDiff,ChampDiff,NQualifiedDiff,AvgWonDiff
0,0.5,2014,1107,1110,5.533333,1.6,1.0,15,6.250000,1.375,0.0,16,16,15,1,-0.716667,1.0,-1,0.225
1,0.5,2014,1107,1112,5.533333,1.6,1.0,15,8.100000,0.400,0.0,10,16,1,15,-2.566667,1.0,5,1.200
2,0.5,2014,1107,1113,5.533333,1.6,1.0,15,15.000000,0.000,0.0,1,16,10,6,-9.466667,1.0,14,1.600
3,0.5,2014,1107,1124,5.533333,1.6,1.0,15,15.000000,0.500,0.0,2,16,6,10,-9.466667,1.0,13,1.100
4,0.5,2014,1107,1140,5.533333,1.6,1.0,15,12.333333,0.000,0.0,3,16,10,6,-6.800000,1.0,12,1.600


In [18]:
#X_test = df_test[['SeedDiff','AvgSeedDiff','ChampDiff', 'NQualifiedDiff', 'AvgWonDiff']]
X_test = df_test[features]
X_test.head()

,SeedDiff
0,1
1,15
2,6
3,10
4,6


In [19]:
# Based on our trained model, predict the probability of winning for the first team
# The [:,1] means we are looking for winning probability, 0 would mean for losing

preds = clf.predict_proba(X_test)[:,1]

clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample.Pred = preds
df_sample


,ID,Pred
0,2014_1107_1110,0.459448
1,2014_1107_1112,0.080327
...,...,...
9110,2017_1457_1462,0.419435
9111,2017_1458_1462,0.619525


In [170]:
#Create the file to be uploaded on Kaggle

df_sample.to_csv('submission4.csv', index=False)